In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA,TruncatedSVD,NMF
from sklearn.preprocessing import Normalizer
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

In [2]:
constituents=pd.read_csv("/data/dharp/compounding/datasets/words.csv")
constituents.columns=['joiner','context','decade','count']
constituents=constituents.query('decade != 2000')
constituents=constituents.groupby(['joiner','decade','context'])['count'].sum()
display(constituents.shape)
constituents.head()

(288777470,)

joiner  decade  context
a_n     1800    a_a         15
                a_n        128
                aa_n        20
                aaa_n       16
                aaron_n      6
Name: count, dtype: int64

In [3]:
compounds=pd.read_csv("/data/dharp/compounding/datasets/phrases.csv")
compounds.columns=['modifier','head','context','decade','count']
compounds=compounds.query('decade != 2000')
compounds['joiner']=compounds['modifier']+" "+compounds['head']
compounds=compounds.groupby(['joiner','decade','context'])['count'].sum()
display(compounds.shape)
compounds.head()

(58614268,)

joiner      decade  context
----_n x_n  1920    x_n         3
            1930    x_n         6
            1940    x_n        36
            1950    x_n        24
            1960    x_n        57
Name: count, dtype: int64

In [4]:
df=pd.concat([constituents,compounds])

In [ ]:
df=df.to_sparse()

In [ ]:
df, rows, columns = df.to_coo(row_levels=['joiner','decade'],column_levels=['context'],sort_labels=False)

In [ ]:
len(columns)

In [ ]:
df_svd = TruncatedSVD(n_components=300, n_iter=10, random_state=1991)
display(df_svd.fit(df).explained_variance_ratio_.sum())
df_list=df_svd.fit(df).explained_variance_ratio_
df_reduced = df_svd.fit_transform(df)
df_reduced = Normalizer(copy=False).fit_transform(df_reduced)
df_reduced=pd.DataFrame(df_reduced,index=rows)
df_reduced.reset_index(inplace=True)
df_reduced

In [ ]:
df_reduced.index = pd.MultiIndex.from_tuples(df_reduced.index, names=['joiner', 'decade'])

In [ ]:
pd.Series(co_occ_list).cumsum().plot(figsize=[15,15])

In [12]:
co_occ_reduced.reset_index(inplace=True)

In [15]:
heads=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_h')]
heads['constituent']=heads['constituent'].str.replace(r'_h$', r'_n', regex=True)
heads.to_csv('/data/dharp/compounding/datasets/heads_context_decadal_svd.csv',index=False,header=False,sep='\t')

In [16]:
modifiers=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_m')]
modifiers['constituent']=modifiers['constituent'].str.replace(r'_m$', r'_n', regex=True)
modifiers.to_csv('/data/dharp/compounding/datasets/modifiers_context_decadal_svd.csv',index=False,header=False,sep='\t')